<a href="https://colab.research.google.com/github/mjag7682/CS9-1-NLP-for-Twitter-Data-for-predicting-stocks/blob/FinALBERT/FinALBERT_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras
!pip install tensorflow
!pip install transformers
!pip3 install albert-tensorflow
!pip install torch
!pip install sentencepiece

     |████████████████████████████████| 1.3MB 9.0MB/s 
     |████████████████████████████████| 1.1MB 50.1MB/s 
     |████████████████████████████████| 2.9MB 50.5MB/s 
     |████████████████████████████████| 890kB 42.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e660d257e555dfb2ca46c0fbcb2f07e411e156954d1e5fe2569e2863847ae25c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 81kB 5.9MB/s 


In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Data/Combined_percentage_2.2.csv', sep=',')
df = df[df['message'].notna()]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
df

,symbol,message,datetime,user,message_id,Date,Time,label
0,AAPL,peak profit last 6 expired option alerts aapl ...,2020-07-19 09:49:35,1442893,229008387,2020-07-19,09:49:35,1
1,AAPL,aapl jul 17 382 50 calls option volume 144 44 ...,2020-07-19 09:47:26,1442893,229008357,2020-07-19,09:47:26,1
2,AAPL,tsla market true bubble territory profitable c...,2020-07-19 09:01:25,1115913,229007569,2020-07-19,09:01:25,1
3,AAPL,aapl analyzed 26 analysts buy consensus 86 ana...,2020-07-19 08:13:00,47688,229006733,2020-07-19,08:13:00,1
4,AAPL,aapl new article dogs dow august 4 adopt ignore,2020-07-19 07:54:05,1555408,229006403,2020-07-19,07:54:05,1
...,...,...,...,...,...,...,...,...
6450984,WMT,added oct 30 puts vz,2009-09-17 15:06:09,564,578933,2009-09-17,15:06:09,0
6450985,WMT,palm pre could use deal vz holidays 09 perhaps,2009-09-17 14:56:06,256,578846,2009-09-17,14:56:06,0
6450986,WMT,bought little vz 29 99,2009-09-17 14:45:21,20,578746,2009-09-17,14:45:21,0
6450987,WMT,thnx agreed like palm pre needs get 39 conside...,2009-09-15 14:08:45,409,568971,2009-09-15,14:08:45,-1


In [ ]:
# Uncomment for FB
# FB_IPO = '2012-05-21'
df.drop(df[df['Date'] < '2019-12-31'].index, inplace = True) 
df


,symbol,message,datetime,user,message_id,Date,Time,label
0,AAPL,peak profit last 6 expired option alerts aapl ...,2020-07-19 09:49:35,1442893,229008387,2020-07-19,09:49:35,1
1,AAPL,aapl jul 17 382 50 calls option volume 144 44 ...,2020-07-19 09:47:26,1442893,229008357,2020-07-19,09:47:26,1
2,AAPL,tsla market true bubble territory profitable c...,2020-07-19 09:01:25,1115913,229007569,2020-07-19,09:01:25,1
3,AAPL,aapl analyzed 26 analysts buy consensus 86 ana...,2020-07-19 08:13:00,47688,229006733,2020-07-19,08:13:00,1
4,AAPL,aapl new article dogs dow august 4 adopt ignore,2020-07-19 07:54:05,1555408,229006403,2020-07-19,07:54:05,1
...,...,...,...,...,...,...,...,...
6437531,WMT,verizon vz business launches comeback coach,2020-07-20 13:51:05,1765977,229194849,2020-07-20,13:51:05,0
6437532,WMT,vz verizon media enables grocery shopping emai...,2020-07-20 13:50:00,47688,229194314,2020-07-20,13:50:00,0
6437533,WMT,vz new article verizon business launches comeb...,2020-07-20 13:45:56,1555408,229192509,2020-07-20,13:45:56,0
6437534,WMT,vz verizon business launches comeback coach,2020-07-20 13:33:55,253206,229187336,2020-07-20,13:33:55,0


In [ ]:
df["label"].value_counts()

 1    616875
-1    454983
 0    167821
Name: label, dtype: int64

In [ ]:
df = df.sample(frac=1)

In [ ]:
df.drop(df[df['label'] == 0].index, inplace = True) 
df["label"].replace({-1: 0}, inplace=True) 

In [ ]:
df["label"].value_counts()

1    616875
0    454983
Name: label, dtype: int64

In [ ]:
df

,symbol,message,datetime,user,message_id,Date,Time,label
62648,AAPL,nflx red amzn aapl fb spy green earnings seaso...,2020-04-22 13:36:13,226994,207828255,2020-04-22,13:36:13,1
3611284,QQQ,spy iwm qq dia quot meet world 39 worst financ...,2020-03-10 18:34:38,673259,199402337,2020-03-10,18:34:38,1
2852618,NFLX,nflx take off rocket rocket rocket bye bears w...,2020-04-22 13:27:01,2917677,207825338,2020-04-22,13:27:01,0
1290052,AMZN,amzn currently trading near 52 week high good ...,2020-05-26 18:40:00,47688,215044213,2020-05-26,18:40:00,0
4535492,TSLA,amzn think reason something far sinister south...,2020-04-21 23:34:51,1900305,207748990,2020-04-21,23:34:51,0
...,...,...,...,...,...,...,...,...
4625418,TSLA,tsla massive massive growth ahead yes right li...,2020-03-11 16:04:49,1021809,199612325,2020-03-11,16:04:49,0
1930206,DIA,09 23 06 angola pledge end opec cheating said ...,2020-07-03 13:23:10,3358599,225188297,2020-07-03,13:23:10,1
4710941,TSLA,tsla tesla market cap 140bn projected 2020 sal...,2020-02-12 19:32:47,79156,194600992,2020-02-12,19:32:47,0
4466221,TSLA,tsla 900 friday,2020-05-20 17:58:06,1922816,214013527,2020-05-20,17:58:06,1


In [ ]:
train_data = df[:1000000]
test_data = df[1000001:1071858]

In [ ]:
print("Number of training examples {}".format(len(train_data)))
# num_examples = 100000
train = train_data.message.values
labels = train_data.label.values
# train = train_data[]

Number of training examples 1000000


In [ ]:
# Print the original sentence.
print(' Original: ', train[10])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(train[10]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train[10])))

 Original:  tsla einhorn loses
Tokenized:  ['▁t', 's', 'la', '▁e', 'in', 'horn', '▁loses']
Token IDs:  [104, 12, 5249, 180, 258, 6901, 3060]


##Testing

###Load Fine-tuned Model

In [ ]:
from transformers import AlbertTokenizer
from transformers.modeling_albert import AlbertModel, load_tf_weights_in_albert, AlbertPreTrainedModel
from transformers import AlbertForSequenceClassification,AlbertConfig
from transformers.tokenization_bert import BertTokenizer
import torch.nn as nn
from torch.nn import CrossEntropyLoss
VOCAB_FILE = "/content/gdrive/My Drive/AGnewsmodel/30k-clean.model" # This is the vocab file output from Build Vocab step
CONFIG_FILE = "/content/gdrive/My Drive/AGnewsmodel/config.json"
tokenizer = AlbertTokenizer(vocab_file=VOCAB_FILE)
config = AlbertConfig.from_json_file(CONFIG_FILE)

PRE_TRAINED_MODEL_NAME_OR_PATH = '/content/gdrive/My Drive/AGnewsmodel/Fine-Tuned'
model = AlbertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME_OR_PATH, num_labels = 2,output_attentions = False, output_hidden_states = False)

# Tell pytorch to run this model on the GPU.
model.cuda()


AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(20001, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [ ]:
print(test_data.message.apply(lambda x: len(x)).quantile([0.9]))
MAX_LEN = 155

0.9    155.0
Name: message, dtype: float64


In [ ]:
test_data.label.value_counts()

1    41403
0    30454
Name: label, dtype: int64

###Convert Text for FinALBERT model Input

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Report the number of sentences.
print('Number of test examples: {}'.format(len(test_data)))
# test_data
# Create sentence and label lists
test = test_data.message.values
labels = test_data.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for text in test:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        text,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LEN,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation = True
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 16  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test examples: 71857


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


###Prediction on test set


In [ ]:
import torch.nn.functional as F  #for softmax function    

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
predicted_labels = []
prediction = np.empty((0,2)) #empty numpy for appending our output
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, attention_mask=b_input_mask)
      # print(outputs)
      pt_predictions = F.softmax(outputs[0], dim=-1)  #applying softmax activation function
      # print(pt_predictions)
      prediction = np.append(prediction, pt_predictions.detach().cpu().numpy(), axis=0) #appending the prediction
      # print(outputs[0][0])
      # predicted_labels.append(np.multiply(np.argmax(outputs[0].detach().cpu().numpy()), b_input_mask.detach().cpu().numpy()))
  logits = outputs[0]
  
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
  # f_predicted_labels = np.concatenate(predicted_labels).astype(int)
  
print('    DONE.')

Predicting labels for 71,857 test sentences...
    DONE.


###Convert model predictions to labels

In [ ]:
prediction.shape
sub = np.argmax(prediction, axis=1)
print(sub)

[1 1 1 ... 1 1 1]


###Add predicted labels to dataframe

In [ ]:
test_data['predicted_labels'] = list(sub)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
test_data

,symbol,message,datetime,user,message_id,Date,Time,label,predicted_labels
2719765,HD,spy careful bulls still asia amp eu need respo...,2020-04-20 18:55:28,543250,207432134,2020-04-20,18:55:28,0,1
4542299,TSLA,tsla 800 today given leggo,2020-04-17 14:55:17,1651138,206977690,2020-04-17,14:55:17,1,1
4852592,TSLA,tsla descending triangle forming,2020-01-10 17:29:31,227500,189908208,2020-01-10,17:29:31,0,1
4647903,TSLA,tsla brokerage houses buying today,2020-03-02 15:16:48,1134428,197676977,2020-03-02,15:16:48,1,1
4625249,TSLA,tsla smoke mirrors joke headed south,2020-03-11 16:39:46,1550829,199622789,2020-03-11,16:39:46,0,1
...,...,...,...,...,...,...,...,...,...
4399117,TSLA,tsla doubt little retail scrubs face tears joy...,2020-07-02 13:02:01,2187451,224867099,2020-07-02,13:02:01,1,1
2840850,NFLX,nflx crossed 20 day moving average macd bullis...,2020-06-09 21:28:19,2843,218526260,2020-06-09,21:28:19,1,1
4522142,TSLA,capr forget tsla play day hands,2020-04-29 17:12:13,2264337,209319820,2020-04-29,17:12:13,1,1
3588982,QQQ,spy qq iwm uso blog regular updates free analy...,2020-04-06 18:57:12,440416,204911533,2020-04-06,18:57:12,1,1


###Classification Report

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

print("F1 score: {}".format(precision_recall_fscore_support(flat_predictions, flat_true_labels, average='macro')))

F1 score: (0.5, 0.2880930180775707, 0.36555712519865796, None)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(flat_true_labels, flat_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     30454
           1       0.58      1.00      0.73     41403

    accuracy                           0.58     71857
   macro avg       0.29      0.50      0.37     71857
weighted avg       0.33      0.58      0.42     71857



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy: {}".format(accuracy_score(flat_predictions, flat_true_labels)))

Accuracy: 0.5761860361551414
